# Deploy COA Objects in Transcend:   Top_Users

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [2]:
%connect Transcend

Password ············


Success: 'Transcend' connection established


In [3]:
database adlste_coa;

Success: 1 rows affected

In [15]:
Drop Table adlste_coa_stg.stg_dat_top_users

Success: 39 rows affected

In [16]:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_top_users
(
   Site_ID                     VARCHAR(64) CHARACTER SET LATIN NOT CASESPECIFIC
 , WeekID                      INTEGER
 , MonthID                     INTEGER
 , UserName                    VARCHAR(256) CHARACTER SET UNICODE 
 , UserHash                    VARCHAR(48)  CHARACTER SET LATIN
 , User_Bucket                 VARCHAR(256) CHARACTER SET LATIN 
 , User_Department             VARCHAR(256) CHARACTER SET LATIN 
 , User_SubDepartment          VARCHAR(256) CHARACTER SET LATIN 
 , User_Region                 VARCHAR(256) CHARACTER SET LATIN 
 , Query_Cnt                   BIGINT
 , Query_Cnt_Rank              INTEGER
 , Query_Complexity_Score      BIGINT
 , Query_Complexity_Score_Rank INTEGER
 , CPU_Sec                     DECIMAL(18,4)
 , CPU_Sec_Rank                INTEGER
 , IOGB                        BIGINT
 , IOGB_Rank                   INTEGER
 , Runtime_Sec                 DECIMAL(18,4)
 , Runtime_Sec_Rank            INTEGER
 , Error_Cnt                   INTEGER
 , Error_Cnt_Rank              INTEGER
 , Combined_Score              INTEGER
 , Total_Rank                  INTEGER
 ) 
 no Primary Index 
 on commit preserve rows
;

Success: 0 rows affected

In [20]:
CREATE MULTISET TABLE adlste_coa.coat_dat_top_users
(
   Site_ID                     VARCHAR(64) CHARACTER SET LATIN NOT CASESPECIFIC
 , WeekID                      INTEGER
 , MonthID                     INTEGER
-- , UserName                    VARCHAR(256) CHARACTER SET UNICODE 
 , UserHash                    VARCHAR(48)  CHARACTER SET LATIN
 , User_Bucket                 VARCHAR(256) CHARACTER SET LATIN 
 , User_Department             VARCHAR(256) CHARACTER SET LATIN 
 , User_SubDepartment          VARCHAR(256) CHARACTER SET LATIN 
 , User_Region                 VARCHAR(256) CHARACTER SET LATIN 
 , Query_Cnt                   BIGINT
 , Query_Cnt_Rank              INTEGER
 , Query_Complexity_Score      BIGINT
 , Query_Complexity_Score_Rank INTEGER
 , CPU_Sec                     DECIMAL(18,4)
 , CPU_Sec_Rank                INTEGER
 , IOGB                        BIGINT
 , IOGB_Rank                   INTEGER
 , Runtime_Sec                 DECIMAL(18,4)
 , Runtime_Sec_Rank            INTEGER
 , Error_Cnt                   INTEGER
 , Error_Cnt_Rank              INTEGER
 , Combined_Score              INTEGER
 , Total_Rank                  INTEGER
 , Run_ID                       INTEGER
 ) 
 Primary Index (Site_ID, MonthID) ;

Success: 0 rows affected

In [21]:
REPLACE PROCEDURE adlste_coa.sp_dat_top_users ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_top_users';
    SET callingsp = 'adlste_coa.sp_dat_top_users';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,current_date as StartDate
    ,current_date as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_top_users
    Group by  1,2,3
    ;
    
    /* end setup audit process */
    
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START') ;

    DELETE FROM adlste_coa.coat_dat_top_users WHERE (Site_ID, coalesce(WeekID,0), coalesce(MonthID,0) ) IN
    (Select Site_ID, coalesce(WeekID,0), coalesce(MonthID,0) From adlste_coa_stg.stg_dat_top_users );
    
    Insert into adlste_coa.coat_dat_top_users 
    Select 
      Site_ID                     
    , WeekID                      
    , MonthID                     
    -- , UserName                    
    , UserHash                    
    , User_Bucket                 
    , User_Department             
    , User_SubDepartment          
    , User_Region                 
    , Query_Cnt                   
    , Query_Cnt_Rank              
    , Query_Complexity_Score      
    , Query_Complexity_Score_Rank 
    , CPU_Sec                     
    , CPU_Sec_Rank                
    , IOGB                        
    , IOGB_Rank                   
    , Runtime_Sec                 
    , Runtime_Sec_Rank            
    , Error_Cnt                   
    , Error_Cnt_Rank              
    , Combined_Score              
    , Total_Rank
    , :runid
    from adlste_coa_stg.stg_dat_top_users ; 
    
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END') ;

END;

Success: 0 rows affected